In [1]:
import os

In [2]:
%pwd

'/Users/bhikipallai/Desktop/Projects/95Mobiles/research/bike'

In [3]:
os.chdir('../../')

In [4]:
%pwd

'/Users/bhikipallai/Desktop/Projects/95Mobiles'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class BikeModelTuningConfig:
    root_dir: Path
    X_train: Path
    X_test: Path
    y_train: Path
    y_test: Path
    tunned_model: Path
    n_estimators: int
    bootstrap: bool
    criterion: str
    oob_score: bool
    mlflow_uri: str

In [6]:
from src.bike.constants import *
from src.bike.utils.common import read_yaml,create_directory

[2024-09-28 23:29:44,047: INFO]: Logging file start


In [7]:
class ConfigurationManger:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directory([self.config.data_root])

    def get_model_tunned(self) -> BikeModelTuningConfig:
        config = self.config.hyper_tune
        params = self.params.model

        create_directory([config.root_dir])

        tunning_config = BikeModelTuningConfig(
            root_dir=config.root_dir,
            X_train = config.X_train,
            X_test=config.X_test,
            y_train=config.y_train,
            y_test=config.y_test,
            tunned_model = config.tunned_model,
            n_estimators = params.n_estimators,
            bootstrap = params.bootstrap,
            criterion=params.criterion,
            oob_score=params.oob_score,
            mlflow_uri="https://dagshub.com/Vicky7873/95Mobiles.mlflow"

        )

        return tunning_config

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge,Lasso,ElasticNet

In [9]:
from sklearn.model_selection import GridSearchCV

In [10]:
import mlflow

In [11]:
import pandas as pd

In [12]:
models = {
    "lr":LinearRegression(),
    "rfr":RandomForestRegressor(),
    "dtr":DecisionTreeRegressor(),
    "ridge":Ridge(),
    "lasso":Lasso(),
    "enet":ElasticNet()
}

In [13]:
grid_params = {
    "lr" : {
        "fit_intercept":[False,True]
        # "positive":[False,True]
    },

    "rfr" : {
        "n_estimators":[1,2,3],
        "criterion": ["squared_error", "absolute_error", "friedman_mse", "poisson"],
        "bootstrap" : [True,False],
        "oob_score" : [True,False]
    },

    "dtr" : {
        "criterion": ["squared_error", "absolute_error", "friedman_mse", "poisson"],
        "splitter":["best","random"],
    },

    "ridge":{
        "alpha":[1.0,0.5,1.5,2.0],
        "solver":['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
    },

    "lasso" : {
        "alpha":[1.0,0.5,1.5,2.0],
    },

    "enet" : {
        "alpha":[1.0,0.5,1.5,2.0],
        "l1_ratio" :[0.3,0.4,0.5],
        "selection": ["cyclic","random"]
    }
}

In [14]:
import joblib
import dagshub
class BikeTunning:
    def __init__(self,config:BikeModelTuningConfig):
        self.config = config
        self.best_score = -float('inf')
    
    def tunned_model(self):
        dagshub.init(repo_owner='Vicky7873', repo_name='95Mobiles', mlflow=True)
        X_train = pd.read_csv(self.config.X_train)
        y_train = pd.read_csv(self.config.y_train)
        mlflow.set_registry_uri(self.config.mlflow_uri)
        mlflow.set_experiment('Bike_model_training')
        with mlflow.start_run():
            for model_name, model in models.items():
                train = GridSearchCV(model,param_grid=grid_params[model_name],cv=5)
                train.fit(X_train,y_train)
                mlflow.log_metric(f"{model_name}_best_score", train.best_score_)
                mlflow.log_params({f"{model_name}_best_params": train.best_params_})
                best_score = train.best_score_

                if best_score>self.best_score:
                    self.best_score = best_score
                    joblib.dump(train.best_estimator_,self.config.tunned_model)
                    print("Best Model Type:", type(train.best_estimator_,).__name__)

        # return best_model

In [15]:
try:
    config = ConfigurationManger()
    tun_config = config.get_model_tunned()
    bike_tune = BikeTunning(config=tun_config)
    bike_tune.tunned_model()
except Exception as e:
    raise e

[2024-09-28 23:29:45,733: INFO]: yaml: config/bike_config.yaml loaded successfully
[2024-09-28 23:29:45,735: INFO]: yaml: params/bike_params.yaml loaded successfully
[2024-09-28 23:29:45,736: INFO]: created directory at: data/bike
[2024-09-28 23:29:45,736: INFO]: created directory at: data/bike/bike_tune
[2024-09-28 23:29:46,109: INFO]: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"


Accessing as Vicky7873

[2024-09-28 23:29:46,120: INFO]: Accessing as Vicky7873
[2024-09-28 23:29:46,514: INFO]: HTTP Request: GET https://dagshub.com/api/v1/repos/Vicky7873/95Mobiles "HTTP/1.1 200 OK"


Initialized MLflow to track repo "Vicky7873/95Mobiles"

[2024-09-28 23:29:46,519: INFO]: Initialized MLflow to track repo "Vicky7873/95Mobiles"


Repository Vicky7873/95Mobiles initialized!

[2024-09-28 23:29:46,521: INFO]: Repository Vicky7873/95Mobiles initialized!
Best Model Type: LinearRegression


/Applications/anaconda3/envs/95m/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Applications/anaconda3/envs/95m/lib/python3.12/site-packages/sklearn/ensemble/_forest.py:615: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
/Applications/anaconda3/envs/95m/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Applications/anaconda3/envs/95m/lib/python3.12/site-packages/sklearn/ensemble/_forest.py:615: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to 

Best Model Type: RandomForestRegressor


/Applications/anaconda3/envs/95m/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Applications/anaconda3/envs/95m/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Applications/anaconda3/envs/95m/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Applications/anaconda3/envs/95m/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Applications/anaconda3/envs/95m/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Applications/a